This notebook contains two prompts used to prompt gpt4 to generate feedback in two categories: lower level style feedback and higher level content feedback
Then the user can copy the outputs (it sometimes takes a few generations to get enough prompts or ones you like) into a json file and run other cells to convert it to feedback. Note that for conversion to feedback the effect, scope, and comparison are all filler.

In [1]:
STYLE_PROMPT = """You are a helpful assistant that always closely follows instructions. Your task it to come up with 100 pieces of feedback a human might give to a LLM and a domain that encompasses all subjects this feedback could be applied to. The feedback should be mainly about the style or formatting of a response instead of broad instructions, and should be easily measurable. The feedback should be very specific, only applicable to certain use cases, and should probably contain some qualifier. The domain should be the same as the qualifier in the feedback and essentially the use case. Keep the domain simple and broad.

Below are a few examples:
Example 1: { "feedback": "Use London gangster vernacular when texting my friend Peter", "domain": "sending texts to my friend Peter" }
Example 2: { "feedback": "Be more detailed in your emails to my PI Anna", "domain": "writing an email to my PI Anna" }
Example 3: { "feedback": "Be more concise when emailing my boss Jared", "domain": "writing an email to my boss Jared" }
Example 4: { "feedback": "End work emails with “Best,\nMoritz”", "domain": "writing work emails" }
Example 5: { "feedback": "Use the British spelling for the word “colour”", "domain": "any situation where the word "color" comes up" }

Be creative and think out of the box. Do not repeat feedback and domains. Output ONLY the feedback and domain in structured json format as a numbered list.
"""

CONTENT_PROMPT = """You are a helpful assistant that always closely follows instructions. Your task it to come up with 100 pieces of feedback a human might give to a LLM and a domain that encompasses all subjects this feedback could be applied to. The feedback should be mainly about the content or tone of a response. The feedback should be very specific, only applicable to certain use cases, and should probably contain some qualifier. The domain should be the same as the qualifier in the feedback and essentially the use case. Keep the domain simple and broad.

Below are a few examples: 
Example 1: 
{ "feedback": "When explaining math questions, assume that I do not have any knowledge of math beyond a fourth grade level", "domain": "Expalining math questions" } 
Example 2: 
{ "feedback": "Whenever you do creative writing, the stories are way too cheery. Please make sure that some stories have dramatic, negative endings going forward", "domain": "Doing creative writing" } 
Example 3: 
{ "feedback": "When writing children's books, use simple language and sentences", "domain": "writing children's books" } 
Example 4: 
{ "feedback": "When texting my mom, use a refined and controlled tone", "domain": "texting my mom" } 
Example 5: 
{ "feedback": "When emailing my boss Jared, use a formal and professional tone", "domain": "emailing Jared }

Be creative and think out of the box. Do not repeat feedback and domains. Output the feedback and domain in structured json format. Output ONLY the json with no other surrounding text.
"""

In [3]:
from openai import OpenAI
client = OpenAI(
    api_key="PUT KEY HERE",
)

In [4]:
def get_response():
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {'role': 'system', 'content': "You are a helpful assistant that always closely follows instructions."},
            {'role': 'user', 'content': CONTENT_PROMPT}
        ],
    )
    response = completion.choices[0].message.content
    return response

In [6]:
import json

## if it's unable to decode its fine to just copy and paste the response into the json file

generated = []
while len(generated) < 100:
    print("num generated", len(generated))
    resp = get_response()
    print(resp)
    if resp[0:7] == "```json": # tries to output in markdown
        resp = resp[8:-3]
    try:
        obj = json.loads(resp)
        generated.extend(obj)
    except json.JSONDecodeError:
        pass

with open("working_json/gpt_feedback_content.json", "w+") as f:
    json.dump(generated, f)

num generated 0
```json
[
  { "feedback": "When discussing historical events, always provide the date and location for context", "domain": "Discussing historical events" },
  { "feedback": "For science fiction narratives, include at least one unexpected twist", "domain": "Writing science fiction" },
  { "feedback": "In DIY instructions, list alternatives for tools that may not be commonly available", "domain": "Providing DIY instructions" },
  { "feedback": "When giving travel advice, suggest activities that are family-friendly", "domain": "Giving travel advice" },
  { "feedback": "On health topics, warn me if you're about to describe something that might be considered graphic or disturbing", "domain": "Discussing health topics" },
  { "feedback": "In coding tutorials, use comments generously to explain the logic", "domain": "Writing coding tutorials" },
  { "feedback": "While summarizing books, always mention the genre and the author", "domain": "Summarizing books" },
  { "feedback": 

In [7]:
# convert json into feedback

import json

ending = "style" # content or style
with open(f"working_json/gpt_feedback_{ending}.json", "r") as f:
    obj = json.load(f)

with open(f"generated_out/gpt_gen_{ending}.py", "w+") as out:
    out.write(f"from dataset.feedback import *\n\ngpt{ending}_feedback = [\n")
    for i in obj:
        out_str = """ 
    Feedback(
        content="{feedback}",
        domain="{domain}",
        effect="placeholder to fix ValidationError",
        scope=Scope.regional,
        categories=["gpt_{ending}"],
        type=Type.qualitative,
        comparison=Comparison.greater_than
    ),
""".format(feedback=i["feedback"], domain=i["domain"], ending=ending)
        out.write(out_str + "\n")
    out.write("]\n")